<a href="https://colab.research.google.com/github/JMindpalace/Machine_Learning/blob/main/Example/004.%EC%98%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>